In [1]:
from tensorflow import keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import copy
import numpy as np
import dlib

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
model = keras.models.load_model('C:/Users/Moh.Massoud/ML/Engagment level/DAiSEE/engagement86.h5')

In [4]:
cap = cv2.VideoCapture(1) # Check if the webcam is opened correctLy 
if not cap.isOpened():
    cap = cv2.VideoCapture('C:/Users/Moh.Massoud/ML/Engagment level/DAiSEE/hafezdemo.mp4') 
if not cap.isOpened():
    raise IOError("Cannot open webcam")
    
    
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

size = (frame_width, frame_height)
result = cv2.VideoWriter('hafez demo.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         25, size)

detector = dlib.get_frontal_face_detector()
font = cv2.FONT_HERSHEY_SIMPLEX

frame_count = 0
state = 0
printState = -1
count = 0
while True:
    _, frame = cap.read()
    count += 1
    if _:
        frame_count += 1
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = detector(gray)
        #predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

        engage=0
        roi=[]
        for face in faces:
            x1 = face.left()
            y1 = face.top()
            x2 = face.right()
            y2 = face.bottom()
            # width = x2 - x1
            # height = y2 - y1
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            roi.append(cv2.resize(cv2.cvtColor(frame[y1:y2, x1:x2], cv2.COLOR_BGR2GRAY), (48,48)))
        predictions = []
        if(len(roi)>0):
            test_images = np.expand_dims(roi, axis=3)
            predictions = model.predict(test_images)
        predict = 0        
        for i in range(len(predictions)):
            predict=np.argmax(predictions[i])
        state += predict

        if frame_count > 25:
            printState = round(state/25,2)
            state = 0
            frame_count = 0

        if printState != -1 and len(roi):    
            cv2.putText(frame, f"{'Engaged' if printState else 'Not Engaged'}", (x1, y1), font, 0.5, (0, 0, 255), 2, cv2.LINE_4)

        cv2.imshow("Drowsiness detection", frame)
        result.write(frame)
    if cv2.waitKey(2) & 0xFF == ord('q'): 
        cap. release() 
        cv2.destroyAllWindows() 
        break

In [5]:
count

3380